## 예측 애널리틱스 : 부스팅

# ■ 실습 목표
      여러가지 부스팅 모델을 구현하고 예측 성능 및 학습 상태를 관찰
      
  ※ 부스팅
  * 앙상블 계열의 모델로 여러 모델의 예측 결과를 종합하여 단일 모델보다 예측력이 우수함
  * 약한 분류기(Weak learner)들을 활용하여 모델 성능을 점진적으로 개선시키는 모델
 
 ### 1. 모듈 및 데이터 불러오기
 
 #### 1.1 모듈불러오기

In [3]:
# 데이터 전처리
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# 기계학습 모델 생성, 학습 ,평가
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier # 패키지를 설체하세요 ~ [pip install lightgbm]
from catboost import CatBoostClassifier # 패키지를 설치하세요 ~ [pip install catboost]
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import torch
from sklearn.metrics import accuracy_score
device = torch.cuda.is_available() # Catboost와 Lightgbm은 gpu 사용이 가능합니다 ! cuda 설치가 되어있다면 활용하면 좋습니다!

# 시각화 & 편의용
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from tqdm import tqdm
import pickle
import warnings
warnings.filterwarnings('ignore')
matplotlib.rcParams['axes.unicode_minus'] = False

# 한글 폰트 설정
plt.rc('font', family='Malgun Gothic')

ModuleNotFoundError: No module named 'lightgbm'

#### 1.2 Wine 데이터 : 와인에 대한 화학적 성분들을 기반하여 와인의 품질의 예측

##### 설명변수 및 반응변수
 * fixed acidity - 고정산, 와인의 산도와 연관됨 (연속형)
 * volatile acidity - 휘발산, 와인의 향과 연관됨(연속형)
 * critic acid - 구연산, 와인의 신선함을 올림(연속형)
 * residual sugar - 잔여 당분, 와인의 단 맛을 올림 (연속형)
 * chlorides - 염화물, 와인의 짠 맛의 원인(연속형)
 * free sulfur dioxide - 황 화합물, 와인을 오래 보관하게 함(연속형) 
 * total sulfur dioxide - 황 화합물, 와인을 오래 보관하게 함(연속형)
 * density - 밀도, 와인의 무게감을 나타냄(연속형) 
 * pH - 산성도, 와인의 신 맛의 정도(연속형)
 * sulphates - 황 화합물, 와인을 오래 보관하게 함(연속형) 
 * alcohol - 알코올, 와인의 단 맛과 무게감에 영향(연속형)
 * quality - 와인등급[타겟변수](10 class 범주형)

In [ ]:
url = 'https://s3.amazonaws.com/assets.datacamp.com/production/course_1606/datasets/winequality-red.csv'
data = pd.read_csv(url, sep=';')
X, y = data.drop(['quality'],axis=1) , data['quality']

In [ ]:
y

### 2 여러가지 부스팅 모델에 대한 학습 방법 및 하이퍼파라미터 소개

#### 2.1 하이퍼 파라미터 소개

# ■ 하이퍼파라미터 소개
      부스팅 모델의 과적합 발생 가능성을 제어하기 위해 하이퍼파라미터 조절은 필수임
      
 ※ 과적합 방지를 위해 주로 탐색하는 하이퍼파라미터
   * Learning rate : 부스팅에 대한 Regulation(Shrinkage) 효과를 지님  
   * N_estimators : 부스팅에 활용될 예측 모델의 총 개수
   * Subsample : 다음 시점의 약 분류기가 학습할 데이터의 비율을 의미하며 0과 1사이의 값을 가짐
   * Max depth : 약한 분류기 모델의 복잡도를 결정하는 값으로 주로 작은 값(1~5)에서 결정됨
   
#### 2.2 모델 특징 소개

# ■ 모델 특징 소개

※ Adaboost
  * 이전 분류기가 예측하지 못한 데이터에 높은 가중치를 둠으로써 다음 차례의 모델은 예측할수 있도록 설계된 모델
 
※ GBM
  * 이전 분류기가 예측하지 못한 에러(gradient)를 다음 차례의 모델이 학습하도록 설계된 모델
  
※ Light GBM
 * 분류기 모델이 의사결정 규칙을 찾을 때, 모든 feature와 data index에 대해 탐색하는 비효율성을 개선
 ▶ Exclusive feature bundling과 Gradient based one side sampling로 비효율성 개선
 
※ Catboost
 * 범주형 변수가 많을 때 효율적이며 Target leakage와 prediction shift 문제점을 해결
 ▶ Exclusive feature bundling과 Gradient based one side sampling로 비효율성 개선
 
#### 2.3 학습

In [ ]:
## 탐색하고자 하는 하이퍼파라미터 설정 ##
model_params = {"GBM": {'model': GradientBoostingClassifier(random_state=0),
                       'params': {'learning_rate':[0.2*(i+1) for i in range(3)],
                                 'subsample':[0.5,0.75,1.0],
                                 'max_depth': [i for i in range(1,11,2)]}},
               "AdaBoost": {'model': AdaBoostClassifier(random_state=0),
                       'params': {'base_estimator':[DecisionTreeClassifier(max_depth=i) for i in range(1,11,2)],
                                 'n_estimators':[50*(i+1) for i in range(3)],
                                 'learning_rate':[0.2*(i+1) for i in range(3)]}},
                "LightGBM": {'model': LGBMClassifier(random_state=0, silent=True,device='gpu') if device == True else LGBMClassifier,
                       'params': {'max_depth':[i for i in range(1,11,2)],
                                 'n_estimators':[50*(i+1) for i in range(3)],
                                 'learning_rate':[0.2*(i+1) for i in range(3)]}},
               "Catboost": {'model': CatBoostClassifier(random_seed=0,silent=True, task_type ='GPU',thread_count=5),
                       'params': {'max_depth':[i for i in range(1,11,2)] ,
                                 'n_estimators':[50*(i+1) for i in range(3)],
                                 'learning_rate':[0.2*(i+1) for i in range(3)]
                                 }}
               }

In [ ]:
# seeds = [0 for i in range(10)]
seeds = [0]
pbar = tqdm(total=len(model_params)*len(seeds))
output = pd.DataFrame([])

for seed in seeds:
    
    X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 2, random_state = seed)
    
    scaler = StandardScaler
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    for model_name, v in model_params.items():
        
        pbar.set_description(desc=f"{model_name}")
        model, params = v['model'], v['params']
        gcv = GridSearchCV(estimator=model, param_grid=params, n_jobs=2 if model_name == 'Catboost' else 5, cv=5,scoring='accuracy')
        gcv.fit(X_train_scaled, y_train)
        
        result = pd.DataFrame.from_dict(gcv.cv_results_)
        result['test_accuracy_with_best_hyp'] = accuracy_score(y_pred = gcv.predict(X_test_scaled) , y_true =  y_test)
        result['model_name'] = model_name
        result['seed'] = seed
        output = pd.concat([output,result])
        pbar.update(1)

output.reset_index(drop=True)
output.to_csv(url)

#### 2.4 학습 결과 확인

In [ ]:
output = pd.read_csv('/classification_result.csv',index_col=0)
model_name = 'Catboost' #GBM , AdaBoost, Light GBM, Catboost

for_plot_values = output.loc[ output['model_name'] == model_name,:] #  예측 모델별 결과 확인을 위한 indexing
fig , axs = plt.subplots(nrows=model_params[model_name]['params'],keys(),__len(), nols=1, figsize=(50,80)) # plot configur
fig.suptitle(f'{model_name} performance for each hyperparameter', fontsize=50) # 전체 Plot 제목

for ind, param_key in enumerate(model_params[model_name]['params'].keys()):
    
    param_for_plot = for_plot_values.groupby([f'param_{param_key}'])['mean_test_score'].mean() # 조절 하이퍼파라미터 이외에 다른
    
    axs[ind].bar(height = param_for_plot.values,x = [i for i in range(param_for_plot.values.shape[0])]) # bar plot
    axs[ind].tick_params(axis='both',labelsize=20) # x,y 축 글씨 폰트 설정
    
    axs[ind].set_xticks([i for i in range(param_for_plot.values.shape[0])]) # x 축 ticks 설정
    axs[ind].set_xtickslabels(param_for_plot.index.to_list()) # x 축 ticks 이름 설정
    
    axs[ind].set_title(f'{param_key}',fontsize=50) # 내부 plot 제목

In [ ]:
plt.figure(figsize=(10,10))
output.groupby(['model_name'])['mean_test_score'].max().plot.bar()
plt.title('Predictive performance of models with optimal hyperparameters', fontsize=20)
plt.yticks(fontsize=20)
plt.show()

#### 2.5 하이퍼파라미터 세밀하게 탐색(your option)

In [ ]:
# 추가 하이퍼파라미터 탐색 ( 촘촘하게 )
model_params = {"GBM": {'model': GradientBoostingClassifier(random_state=0),
                       'params': {'learning_rate':list(np.arange(0.05,0.20,0.01)), #0.2 근처 혹은 작을수록 좋은 성능을 보였다
                                 'n_estimators' : [50] , # 너무 많은 모델 개수를 필요로 하지 않아 보였음 (50에서 가장 Best)
                                 'subsample':[1.0], # 1에 가까울수록 좋은 성능을 보였음
                                 'max_depth': [7,8,9] # 7과 9 사이에서 조밀하게 탐색
                                 }},
                
               "AdaBoost": {'model': AdaBoostClassifier(random_state=0),
                       'params': {'base_estimator':[DecisionTreeClassifier(max_depth=i) for i in [7,8,9]], # 7과 9사이에
                                 'n_estimators':[50], # 너무 많은 모델 개수를 필요로 하지 않아 보였음 (50에서 가장 Best)
                                 'learning_rate':list(np.arange(0.05,0.20,0.01))}},
                
                "LightGBM": {'model': LGBMClassifier(random_state=0, silent=True,device='gpu') if device == True else LG~~~~,
                       'params': {'max_depth':[1,2,3],
                                 'n_estimators':[100],
                                 'learning_rate':list(np.arange(0.05,0.20,0.01))}}, # 작을수록 좋은 성능을 보였음
                
               "Catboost": {'model': CatBoostClassifier(random_seed=0,silient=True, task_type ='GPU',thread_count=5,~~~~~),
                       'params': {'max_depth':[8,9,10] ,
                                 'n_estimators':[150],
                                 'learning_rate': list(np.arange(0.05,0.20,0.05)) # 다른 모델보단 성능에 둔감하게 영향을
                                 }}}

# Train
# seeds = [0 for i in range(10)]
seeds = [0]
pbar = tqdm(total=len(model_params)*len(seeds))
output = pd.DataFrame([])

for seed in seeds:
    
    X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 2 , random_state =seed)
    
    scaler = StandardScaler
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    for model_name, v in model_params.items():
        
        pbar.set_description(desc=f"{model_name}")
        model, params = v['model'], v['params']
        gcv = GridSearchCV(estimator=model, param_grid_params, n_jobs=2 if model_name == 'Catboost' else 5, cv=5,scoring='a~~~')
        gcv.fit(X_train_scaled, y_train)
        
        result = pd.DataFrame.from_dict(gcv.cv_results_)
        result['test_accuracy_with_best_hyp'] = accuracy_score(y_pred = gcv.predict(X_test_scaled) , y_true =  y_test)
        result['model_name'] = model_name
        result['seed'] = seed
        output = pd.concat([output,result])
        pbar.update(1)

output.reset_index(drop=True)
output.to_csv('./classification_result.csv')

#### 2.6 테스팅 데이터에 대한 최종 예측 성능 관찰

In [ ]:
output = pd.read_csv('/classification_result.csv',index_col=0)
output.reset_index(drop=True, inplace=True) # index 초기화
plt.figure(figsize=(10,10))
idx = output.groupby(['model_name'])['mean_test_score'].idxmax() # 각 모델별로 검증용 accuracy를 기준하여, Best 성능 기록한
output.loc[idx,['model_name','test_accuracy_with_best_hyp']].set_index('model_name').plot.bar(legend=False)
plt.title('Predictive performance of models with optimal hyperparameters', fontsize=20)
plt.yticks(fontsize=20)
plt.show()

## E.O.D